In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Male\GeneCpG\Depression
data = pd.read_csv('DepressionCateg_Methylome_Male.csv')
data.head(1)

Z:\PPMI_Data\Excels\Only_Male\GeneCpG\Depression


,Unnamed: 0,Sentrix,PATNO,HYS,Gender,APPRDX,MDS_Depress,cg20815302,cg25795414,cg00959375,...,cg02293835,cg25106148,cg09135009,cg16616325,cg08020573,cg19102621,cg26302779,cg10377074,cg06750710,ch.14.849914F
0,0,200991620021_R01C01,3001,2,1,1,0,0.907957,0.86891,0.752844,...,0.890264,0.096523,0.088376,0.035056,0.040317,0.620467,0.917245,0.87378,0.809904,0.045885


In [3]:
X = data.drop(['Unnamed: 0','Sentrix','PATNO','HYS','Gender','MDS_Depress'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7258064516129032

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100


BorutaPy finished running.

Iteration: 	10 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	106975


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x1A1C3E0AE40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1A1C3E0AE40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
0,APPRDX,1
54064,cg10981065,2
13629,cg05817202,3


In [10]:
Top50.to_csv ('Depression_BorutaMale_Top50.csv', index = False)
selected_rf_features.to_csv ('Depression_BorutaMale_All.csv', index = False)

#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
